# HMS and MinIO Connectivity Test

Run these cells to diagnose the connection to Hive Metastore and MinIO.


In [1]:
# Test 1: DNS Resolution
import socket

services = ['hive_metastore', 'minio_hms']

for service in services:
    try:
        ip = socket.gethostbyname(service)
        print(f"✓ DNS Resolution works: {service} -> {ip}")
    except socket.gaierror as e:
        print(f"✗ DNS Resolution FAILED for {service}: {e}")


✓ DNS Resolution works: hive_metastore -> 172.22.0.3
✓ DNS Resolution works: minio_hms -> 172.22.0.2


In [2]:
# Test 2: HMS Thrift Port (9083)
import socket

try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(5)
    result = sock.connect_ex(('hive_metastore', 9083))
    sock.close()
    
    if result == 0:
        print("✓ HMS Thrift port 9083 is accessible")
    else:
        print(f"✗ HMS Thrift port 9083 is NOT accessible (error code: {result})")
except Exception as e:
    print(f"✗ HMS Connection test FAILED: {e}")


✓ HMS Thrift port 9083 is accessible


In [3]:
# Test 3: MinIO HTTP Connectivity
import urllib.request

try:
    response = urllib.request.urlopen('http://minio_hms:9000/minio/health/live', timeout=5)
    print(f"✓ MinIO HTTP Connection works: Status {response.status}")
except Exception as e:
    print(f"✗ MinIO HTTP Connection FAILED: {e}")


✗ MinIO HTTP Connection FAILED: HTTP Error 400: Bad Request


In [4]:
# Test 4: Check /etc/hosts
import subprocess

result = subprocess.run(['cat', '/etc/hosts'], capture_output=True, text=True)
print("Contents of /etc/hosts:")
print(result.stdout)


Contents of /etc/hosts:
127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback
fe00::	ip6-localnet
ff00::	ip6-mcastprefix
ff02::1	ip6-allnodes
ff02::2	ip6-allrouters
172.22.0.4	0736ede68d77



In [5]:
# Test 5: Container Network Information
import subprocess

result = subprocess.run(['hostname', '-i'], capture_output=True, text=True)
print(f"Container IP: {result.stdout.strip()}")

# Show all network interfaces
result = subprocess.run(['ip', 'addr'], capture_output=True, text=True)
print("\nNetwork Interfaces:")
print(result.stdout)


Container IP: 172.22.0.4


FileNotFoundError: [Errno 2] No such file or directory: 'ip'

In [ ]:
# Test 6: Can we resolve and connect via IP?
import socket
import urllib.request

try:
    # Get MinIO IP
    minio_ip = socket.gethostbyname('minio_hms')
    print(f"MinIO IP: {minio_ip}")
    
    # Try connecting via IP
    response = urllib.request.urlopen(f'http://{minio_ip}:9000/minio/health/live', timeout=5)
    print(f"✓ Direct IP connection works: Status {response.status}")
except Exception as e:
    print(f"✗ Direct IP connection FAILED: {e}")
